In [11]:
import numpy as np
import itertools
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

import abus_classification

In [3]:
x = np.zeros((5,5,5), dtype=np.uint8)
x[1:4,1:4,1:4] = np.ones((1,1,1), dtype=np.uint8)
    

In [20]:
directions = np.array(list(itertools.product((-1,0,1), repeat=3)), dtype=np.int32)
boundary_points = []

for i in range(5):
    for j in range(5):
        for k in range(5):
            anchor_value = x[i,j,k]
            if anchor_value != 0:
                print(i,j,k)
                anchor_position = np.array([i,j,k], np.uint)
                for neighbor_direction in directions:
                    neighbor_position = anchor_position + neighbor_direction                
                    a = neighbor_direction[0]
                    b = neighbor_direction[1]
                    c = neighbor_direction[2]
                    
                    if a < 0 or b < 0 or c < 0:
                        continue
                    
                    if x[a,b,c] == 0:
                        boundary_points.append((a,b,c))

print(boundary_points)
                    

1 1 1
1 1 2
1 1 3
1 2 1
1 2 2
1 2 3
1 3 1
1 3 2
1 3 3
2 1 1
2 1 2
2 1 3
2 2 1
2 2 2
2 2 3
2 3 1
2 3 2
2 3 3
3 1 1
3 1 2
3 1 3
3 2 1
3 2 2
3 2 3
3 3 1
3 3 2
3 3 3
[(0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (1, 1, 0), (0, 0, 0), (0, 0, 1), (0, 1, 0), (0, 1, 1), (1, 0, 0), (1, 0, 1), (

In [16]:
a = directions[0][0]
b = directions[1][0]
c = directions[2][0]
print(x[a,b,c])

0
